<a href="https://colab.research.google.com/github/angelaaaateng/DEPA_Project/blob/master/Python_GCP_Master3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!pip install gcsfs
!pip install praw
!pip install psaw

import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import praw
from psaw import PushshiftAPI
import gcsfs
df_names = pd.read_csv('gs://cryptodb-files/crypto_names.csv')

dates_df = pd.DataFrame({'date': pd.date_range(start='1/1/2009', end=date.today()), 'date_id' : range(len(pd.date_range(start='1/1/2009', end=date.today())))})

#dates_df.set_index('date_id', inplace = True)

dates_df.head()


     |████████████████████████████████| 61kB 2.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/9f/0f/b9/5bc5222756d121ccace51ab3084a1c733380908a4e2f939038
Successfully built gcsfs
     |████████████████████████████████| 122kB 2.8MB/s 
     |████████████████████████████████| 204kB 8.9MB/s 


,date,date_id
0,2009-01-01,0
1,2009-01-02,1
2,2009-01-03,2
3,2009-01-04,3
4,2009-01-05,4


In [3]:
df_names.head()

,coin_id,name,symbol,slug
0,0,0x,ZRX,0x
1,1,Aeternity,AE,aeternity
2,2,Aion,AION,aion
3,3,Ardor,ARDR,ardor
4,4,Ark,ARK,ark


In [5]:

# # Reddit Table
# #### Reddit posts from /r/Cryptocurrency from 2017-2019


#Connect to reddit API
reddit = praw.Reddit(client_id='GjfUHQE8AYnXLg', client_secret='PfbhtsXJGAAUNiEyHPRGPuFJ0ro', user_agent='DEPA_Project')
api = PushshiftAPI()

#Extract all posts to /r/CryptoCurrency from 2017-2019

start_epoch=int(dt.datetime(2017, 1, 1).timestamp())
end_epoch = int(dt.datetime(2019, 1, 1).timestamp())

reddit_data = pd.DataFrame(api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit='CryptoCurrency',
                            filter=['author', 'title', 'subreddit', 'num_comments', 'created', 'score']))

#Convert the created_utc column to a date 
reddit_data['created_utc']=(pd.to_datetime(reddit_data['created_utc'],unit='s'))

#remove redundant columns
reddit_data = reddit_data.drop(['created', 'd_'], axis = 1)


#Rename index to post_id
#reddit_data.index.names = ['post_id']

#convert created_utc to just a date
#reddit_data['created_utc'] = reddit_data['created_utc'].dt.date


# reddit_data['created_utc'] = pd.to_datetime(reddit_data['created_utc'])


#reddit_data.head()


#Add date_id column to reddit_data
#reddit_data2 = reddit_data.merge(dates_df, how = 'left', left_on = 'created_utc', right_on = 'date')


#Rename created_utc to date
reddit_data.rename(columns = {'created_utc':'date'}, inplace = True)


#Drop date column
#reddit_data2 = reddit_data2.drop('date', axis = 1)


reddit_data.index.names = ['post_id']

reddit_data.head()


,author,date,num_comments,score,subreddit,title
post_id,,,,,,
0,h1121900,2018-12-31 23:56:47,0,1,CryptoCurrency,Dec 31 2018 Important Crypto News
1,historypaleo,2018-12-31 23:52:25,0,1,CryptoCurrency,Law firm files lawsuit against NVIDIA for fals...
2,LamboJambo,2018-12-31 23:48:35,7,1,CryptoCurrency,My wish for 2019
3,h1121900,2018-12-31 23:45:46,0,1,CryptoCurrency,Dec 31 2018 Important Crypto News
4,h214289,2018-12-31 23:38:15,0,1,CryptoCurrency,Dec 31 2018 Important Crypto News


In [6]:
# # Create reddit_coins Join Table 
# #### Matches the reddit posts in reddit_data table to the coins in coins table


#Zip the names into tuple of (name, slug, symbol)
zipped_names = list(zip(df_names['name'],df_names['slug'],df_names['symbol']))

#Create a search list - seperating the three terms with OR operator (|)
search_list = []
for (name, slug, symbol) in zipped_names:
    listt = [name, slug, symbol]
    pat = '|'.join(listt)
    search_list.append(pat)


#Add boolean columns
import re
dummy_df = pd.DataFrame(dict((name, reddit_data.title.str.contains(name, re.IGNORECASE))
                             for name in search_list))



#Convert dummy columns into rows with post_id as the index
i, j = np.where(dummy_df)

coins_mentioned_series = pd.Series(dict(zip(zip(i, j), dummy_df.columns[j])))



#Create final join table between reddit and coins table. Rename columns of dataframe. 
coin_reddit_join = pd.DataFrame(coins_mentioned_series).reset_index()
coin_reddit_join.columns = ['post_id', 'coin_id', 'coin_name']
coin_reddit_join = coin_reddit_join.drop('coin_name', axis = 1)


coin_reddit_join2 = coin_reddit_join.set_index(['post_id', 'coin_id'],)


coin_reddit_join2


Empty DataFrame
Columns: []
Index: [(5, 11), (5, 77), (7, 0), (8, 77), (10, 4), (10, 79), (11, 11), (12, 4), (16, 31), (16, 77), (17, 77), (22, 11), (23, 4), (25, 35), (27, 35), (27, 77), (30, 77), (32, 11), (32, 77), (33, 4), (33, 11), (33, 77), (34, 63), (35, 4), (35, 25), (35, 77), (36, 77), (37, 25), (38, 11), (40, 4), (44, 4), (44, 79), (45, 4), (45, 11), (48, 11), (48, 77), (50, 11), (52, 11), (54, 77), (55, 77), (60, 4), (60, 11), (62, 11), (62, 77), (64, 4), (72, 4), (75, 77), (76, 4), (79, 11), (81, 77), (84, 67), (84, 77), (85, 4), (86, 19), (89, 4), (89, 77), (94, 35), (96, 77), (96, 96), (97, 77), (98, 46), (106, 11), (108, 11), (109, 11), (109, 77), (110, 11), (110, 77), (112, 77), (113, 74), (115, 77), (116, 11), (120, 44), (121, 11), (123, 4), (123, 11), (134, 11), (134, 15), (134, 77), (134, 83), (135, 11), (136, 4), (141, 77), (144, 4), (144, 11), (147, 4), (152, 17), (152, 77), (153, 35), (154, 44), (155, 46), (157, 77), (160, 77), (161, 11), (162, 77), (166, 77), (168, 11), (168, 77), (170, 4), (172, 25), (172, 67), ...]

[266770 rows x 0 columns]

In [8]:
# # Pricing Table

# In[55]:


import requests
import datetime
import pandas as pd


# In[56]:


def daily_price_historical(symbol, comparison_symbol, all_data=True, limit=1, aggregate=1, exchange=''):
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    if all_data:
        url += '&allData=true'
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    return df


# In[57]:


pricing_list = []

for symbol in list(df_names['symbol']):
# initialise scraper with time interval
    try:
        df = daily_price_historical(symbol, 'USD')
        df['name'] = symbol
        pricing_list.append(df)
    except:
        df2 = pd.DataFrame(columns = df.columns)
        pricing_list.append(df2)


pricing_df = pd.concat(pricing_list)



pricing_df2 = pricing_df.merge(df_names[['symbol', 'coin_id']], how = 'left', left_on = 'name', right_on = 'symbol')
pricing_df3 = pricing_df2.drop(['name', 'symbol', 'time'], axis = 1)


#Convert to date
#pricing_df3['timestamp'] = pricing_df3['timestamp'].dt.date



#Make it a datetime object 
pricing_df3['timestamp'] = pd.to_datetime(pricing_df3['timestamp'])


#Rename timestamp to date
pricing_df3.rename(columns = {'timestamp':'date'}, inplace = True)


#Add date_id column to reddit_data
#pricing_df4 = pricing_df3.merge(dates_df, how = 'left', left_on = 'timestamp', right_on = 'date')

#remove timestamp (replaced with date)
#pricing_df4 = pricing_df4.drop('timestamp', axis = 1)

#pricing_df4['date_id'].astype('int')


# In[391]:


#Drop Redundant date columns
#pricing_df5 = pricing_df4.drop('date', axis = 1)

#pricing_df5.set_index(['date_id', 'coin_id'], inplace = True)


pricing_df3.set_index(['date', 'coin_id'], inplace = True)

# In[392]:


pricing_df3.head()


,,close,high,low,open,volumefrom,volumeto
date,coin_id,,,,,,
2017-08-11,0,0.06687,0.4444,0.06687,0.06950,39437.24,2994.43
2017-08-12,0,0.16250,0.1830,0.06687,0.06687,6232.81,765.38
2017-08-13,0,0.19000,0.3880,0.12500,0.16250,10382.08,1885.11
2017-08-14,0,0.32000,0.3300,0.13100,0.19000,7936.82,1788.88
2017-08-15,0,1.00000,1.8000,0.32000,0.32000,3467.80,2905.58


In [9]:
# # Google Trends table

!pip install pytrends
!pip install cryptory
import cryptory
from cryptory import Cryptory

# initialise object 
# pull data from start of 2017 to present day
my_cryptory = Cryptory(from_date = "2017-01-01",to_date="2019-01-01", ascending=True)

#Pulling google trends data from cryptory 

i=1
google_data_list = []
for name in list(df_names['name']):
    if(i>0 and i<101):
        i=i+1
        kw_list = []
        kw_list.append(name)
        try:
            data = my_cryptory.get_google_trends(kw_list)
            google_data_list.append(data)
        except:
            continue;


trend_df1 = pd.concat(google_data_list)

trend_df2 = pd.DataFrame(trend_df1.pivot_table(index = 'date').unstack()).reset_index()


#Adding coin_id column
trend_df3 = trend_df2.merge(df_names[['name', 'coin_id']], how = 'left', left_on = 'level_0', right_on = 'name')

#Dropping columns
trend_df3 = trend_df3.drop(['level_0', 'name'], axis = 1)

#Rename columns
trend_df3.columns = ['date', 'trend', 'coin_id']



#Add date_id column
#trend_df4 = trend_df3.merge(dates_df, how = 'left', left_on = 'date', right_on = 'date').drop('date', axis = 1)


trend_df3.set_index(['date', 'coin_id'], inplace = True)

trend_df3.head()


  Stored in directory: /root/.cache/pip/wheels/ec/b4/6f/6ae72a8dc1ca028c78d88b90381b34166b2b6d61c99349bf4c
Successfully built cryptory


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,,trend
date,coin_id,
2017-01-01,0,12.958482
2017-01-02,0,8.732890
2017-01-03,0,15.775544
2017-01-04,0,12.676776
2017-01-05,0,16.057250


In [14]:


# # Twitter Table

import re
import csv
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML


# In[182]:


import pandas as pd
tweets_df = pd.DataFrame({'Date': [], 'No. of Tweets': [], 'Coin': []})

for i in range(0,len(df_names['symbol'])-1):
    #len(coin_name["symbol"])-1

    coin = df_names['symbol'][i]
    url = 'https://bitinfocharts.com/comparison/tweets-'+coin.lower()+'.html'
    headers = {'User-Agent': "Chrome/54.0.2840.90"}
    response = requests.get(url, headers=headers)
    html = response.text 

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    x = soup.find_all('script')

    data_1 = re.findall(r'(\[new\sDate.*\]])', str(x))
    data_1 = str(data_1)
    
    
    if data_1 == '[]':
        continue
    data_2 = data_1.split("],[")
    data_2[0] = data_2[0][3:]
    data_2[len(data_2) - 1] = data_2[len(data_2) - 1][:-4]
    data_pd = pd.DataFrame(data_2)
    data_clean = data_pd[0].str.split(",", expand = True)
    data_clean = data_clean.iloc[:, 0:2]
    data_clean.columns = ['Date', 'No. of Tweets']
    data_clean["Date"] = data_clean["Date"].str.slice(10,20)
    data_clean["Coin"] = coin.upper()
    tweets_df = tweets_df.append(data_clean, ignore_index = True)


#Convert Date into datetime object
tweets_df['Date'] = tweets_df['Date'].astype('datetime64')


#Remove the null rows
tweets_df = tweets_df[tweets_df['No. of Tweets'] != 'null']


#Convert No. of Tweets into int object
tweets_df['No. of Tweets'] = tweets_df['No. of Tweets'].astype('int')


#Adding coin_id column
tweets_df = tweets_df.merge(df_names[['symbol', 'coin_id']], how = 'left', left_on = 'Coin', right_on = 'symbol')

#Drop redundant name columns
tweets_df = tweets_df.drop(['Coin', 'symbol'], axis = 1)


#add date_id column
#tweets_df2 = tweets_df.merge(dates_df, how = 'left', left_on = 'Date', right_on = 'date')


#Change to tweet_count
tweets_df.rename(columns = {'No. of Tweets':'tweet_count'}, inplace = True)


# In[ ]:


#tweets_df3 = tweets_df2.drop(['Date', 'date'], axis = 1)


tweets_df.set_index(['Date', 'coin_id'], inplace = True)

tweets_df.head()

,,tweet_count
Date,coin_id,
2018-01-08,0,7
2018-01-09,0,20
2018-01-10,0,21
2018-01-11,0,4
2018-01-12,0,6


In [16]:

# # Reddit Subscribers

#Subreddits based on symbol

empty_df = pd.DataFrame(columns = ['date','total_subscribers','subreddit'])
subreddit_list = []
for name in list(df_names['symbol']):
    try:
        subred = my_cryptory.extract_reddit_metrics(subreddit = name, metric = "total-subscribers", col_label="", sub_col=True)
        if (type(subred) == ValueError):
            subreddit_list.append(empty_df)
        else:
            subreddit_list.append(subred)
    except:
        subreddit_list.append(empty_df)


# In[308]:


#Concatenate DF's of symbol-based subreddits
reddit_subscribers = pd.concat(subreddit_list)


# In[331]:


#Add the date_id column
#reddit_subscribers1 = reddit_subscribers.merge(dates_df, how = 'left', left_on = 'date', right_on = 'date')

#Add the coin_id column
reddit_subscribers2 = reddit_subscribers.merge(df_names[['symbol', 'coin_id']], how = 'left', left_on = 'subreddit', right_on = 'symbol')



#Drop the symbol column
reddit_subscribers3 = reddit_subscribers2.drop(['symbol'], axis = 1)



#Subreddits based on name

empty_df = pd.DataFrame(columns = ['date','total_subscribers','subreddit'])
subreddit_list2 = []
for name in list(df_names['name']):
    try:
        subred = my_cryptory.extract_reddit_metrics(subreddit = name, metric = "total-subscribers", col_label="", sub_col=True)
        if (type(subred) == ValueError):
            subreddit_list2.append(empty_df)
        else:
            subreddit_list2.append(subred)
    except:
        subreddit_list2.append(empty_df)


#Concatenate DF's of name-based subreddits
reddit_subscribers_name = pd.concat(subreddit_list2)


# In[336]:


#Add the date_id column
#reddit_subscribers_name1 = reddit_subscribers_name.merge(dates_df, how = 'left', left_on = 'date', right_on = 'date')

#Add the coin_id column
reddit_subscribers_name2 = reddit_subscribers_name.merge(df_names[['name', 'coin_id']], how = 'left', left_on = 'subreddit', right_on = 'name')


#Drop the name column
reddit_subscribers_name3 = reddit_subscribers_name2.drop('name', axis = 1)


# In[343]:


# CONCATENATE BOTH SUBSCRIBERS TABLES INTO ONE

reddit_subscribers_final = pd.concat([reddit_subscribers3, reddit_subscribers_name3])


# In[372]:


#Drop Redundant date columns
#reddit_subscribers_final2 = reddit_subscribers_final.drop('date', axis = 1)




reddit_subscribers_final.set_index(['date', 'coin_id'], inplace = True)


reddit_subscribers_final.head()

,,total_subscribers,subreddit
date,coin_id,,
2017-01-01,2,2008,AION
2017-01-02,2,2007,AION
2017-01-03,2,2008,AION
2017-01-04,2,2010,AION
2017-01-05,2,2013,AION


In [0]:
from datalab.context import Context
import datalab.storage as gcs
context.default().set_project_id('leafy-stock-240814')


gcs.Bucket('python_output').item('dates_table_final.csv').write_to(dates_df.to_csv(),'text/csv')

gcs.Bucket('python_output').item('reddit_post_final.csv').write_to(reddit_data2.to_csv(encoding='utf-16'),'text/csv')

gcs.Bucket('python_output').item('coin_reddit_final.csv').write_to(coin_reddit_join2.to_csv(),'text/csv')

gcs.Bucket('python_output').item('google_trends_final.csv').write_to(trend_df4.to_csv(),'text/csv')

gcs.Bucket('python_output').item('pricing_final.csv').write_to(pricing_df5.to_csv(),'text/csv')

gcs.Bucket('python_output').item('tweets_final.csv').write_to(tweets_df3.to_csv(),'text/csv')

gcs.Bucket('python_output').item('rsubscribers_final.csv').write_to(reddit_subscribers_final2.to_csv(),'text/csv')





In [27]:

print(reddit_data.head())

print(coin_reddit_join2.head())

print(pricing_df3.head())

print(trend_df3.head())

print(tweets_df.head())

print(reddit_subscribers_final.head())


               author  ...                                              title
post_id                ...                                                   
0            h1121900  ...                  Dec 31 2018 Important Crypto News
1        historypaleo  ...  Law firm files lawsuit against NVIDIA for fals...
2          LamboJambo  ...                                   My wish for 2019
3            h1121900  ...                  Dec 31 2018 Important Crypto News
4             h214289  ...                  Dec 31 2018 Important Crypto News

[5 rows x 6 columns]
Empty DataFrame
Columns: []
Index: [(5, 11), (5, 77), (7, 0), (8, 77), (10, 4)]
                      close    high      low     open  volumefrom  volumeto
date       coin_id                                                         
2017-08-11 0        0.06687  0.4444  0.06687  0.06950    39437.24   2994.43
2017-08-12 0        0.16250  0.1830  0.06687  0.06687     6232.81    765.38
2017-08-13 0        0.19000  0.3880  0.12500  0.

In [28]:
from google.colab import drive
drive.mount('drive')
#After authentication, you can copy your csv file.

reddit_data.to_csv('reddit_data.csv', header = None)
!cp reddit_data.csv drive/My\ Drive/


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
coin_reddit_join2.to_csv('coin_reddit_join2.csv', header = None)
!cp coin_reddit_join2.csv drive/My\ Drive/


pricing_df3.to_csv('pricing_df3.csv', header = None)
!cp pricing_df3.csv drive/My\ Drive/

trend_df3.to_csv('trend_df3.csv', header = None)
!cp trend_df3.csv drive/My\ Drive/

tweets_df.to_csv('tweets_df.csv', header = None)
!cp tweets_df.csv drive/My\ Drive/

reddit_subscribers_final.to_csv('reddit_subscribers_final.csv', header = None)
!cp reddit_subscribers_final.csv drive/My\ Drive/



In [0]:
coin_reddit_join2.to_csv('coin_reddit_join2.csv', header = None)
!cp coin_reddit_join2.csv drive/My\ Drive/